# Exploring different methods of hyperparameter tuning

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
from tensorflow import keras
from scipy.stats import reciprocal

## Loading dataset

In [2]:
housing = fetch_california_housing()

In [3]:
df_data = pd.DataFrame(housing.data, columns = housing.feature_names)

In [4]:
df_data['target'] = pd.Series(housing.target)

In [5]:
df_data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,target
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


## Splitting the data

- We will be splitting the data into train, validation and test sets.

In [6]:
x = np.array(df_data.drop(columns = 'target', axis = 1))
y = np.array(df_data.target)

In [7]:
x_train_full, x_test, y_train_full, y_test = train_test_split(x, y, random_state = 42)
x_train, x_valid, y_train, y_valid = train_test_split(x_train_full, y_train_full, random_state = 42)

## Scaling the data

- Scaling the data using Standard Scaler

In [8]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_valid = scaler.transform(x_valid)

## Building the model

In [9]:
def build_model(n_layers = 1, n_units = 30, input_shape = [8], learning_rate = 3e-3):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape = input_shape))
    for i in range(n_layers):
        model.add(keras.layers.Dense(n_units, activation = 'relu'))
    model.add(keras.layers.Dense(1))
    model.compile(loss = 'mse', optimizer = keras.optimizers.SGD(lr = learning_rate))
    return model

- The above func creates a NN model given a set of hyperparameters.
- It creates a simple Sequential model for univariate-regression, given the no. of hidden layers, no. of units in each layer, the input shape and also the learning rate of the optimizer.

In [10]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

- The KerasRegressor object is a thin wrapper around the Keras model buit using teh func. 
- Now we can use this object like a regular scikit-learn regressor.

## Training the model

In [11]:
keras_reg.fit(x_train, y_train, epochs = 100, validation_data = (x_valid, y_valid), callbacks = [keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True)])

Epoch 1/100
363/363 [==============================] - 0s 1ms/step - loss: 1.1622 - val_loss: 1.3006
Epoch 2/100
363/363 [==============================] - 0s 831us/step - loss: 0.6166 - val_loss: 0.9012
Epoch 3/100
363/363 [==============================] - 0s 774us/step - loss: 0.5362 - val_loss: 0.4949
Epoch 4/100
363/363 [==============================] - 0s 772us/step - loss: 0.4826 - val_loss: 0.4840
Epoch 5/100
363/363 [==============================] - 0s 797us/step - loss: 0.4579 - val_loss: 0.4476
Epoch 6/100
363/363 [==============================] - 0s 817us/step - loss: 0.4365 - val_loss: 0.4388
Epoch 7/100
363/363 [==============================] - 0s 862us/step - loss: 0.4245 - val_loss: 0.4001
Epoch 8/100
363/363 [==============================] - 0s 807us/step - loss: 0.4163 - val_loss: 0.4170
Epoch 9/100
363/363 [==============================] - 0s 828us/step - loss: 0.4101 - val_loss: 0.3885
Epoch 10/100
363/363 [==============================] - 0s 825us/step - los

In [12]:
keras_reg.score(x_test, y_test)

162/162 [==============================] - 0s 558us/step - loss: 0.3815


-0.3815077245235443

- The point to be noted here is that the score will be opposite of the MSE because scikit-learn wants scores, not losses(therefore higher should be better). 

## Using RandomizedSearchCV

- Since there are may hyperparameters it is preferrable to use randomized search rather than grid search .

In [13]:
params = {'n_layers' : [0, 1, 2, 3], 'n_units' : np.arange(1, 100), 'learning_rate' : reciprocal(3e-4, 3e-2)}

In [14]:
rand_search_cv = RandomizedSearchCV(keras_reg, params, n_iter = 10, cv = 3)

In [15]:
rand_search_cv.fit(x_train, y_train, epochs = 100, validation_data = (x_valid, y_valid), callbacks = keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True))

Epoch 1/100
242/242 [==============================] - 0s 1ms/step - loss: 1.0074 - val_loss: 299.8259
Epoch 2/100
242/242 [==============================] - 0s 845us/step - loss: nan - val_loss: nan
Epoch 3/100
242/242 [==============================] - 0s 909us/step - loss: nan - val_loss: nan
Epoch 4/100
242/242 [==============================] - 0s 894us/step - loss: nan - val_loss: nan
Epoch 5/100
242/242 [==============================] - 0s 872us/step - loss: nan - val_loss: nan
Epoch 6/100
242/242 [==============================] - 0s 755us/step - loss: nan - val_loss: nan
Epoch 7/100
242/242 [==============================] - 0s 842us/step - loss: nan - val_loss: nan
Epoch 8/100
242/242 [==============================] - 0s 870us/step - loss: nan - val_loss: nan
Epoch 9/100
242/242 [==============================] - 0s 883us/step - loss: nan - val_loss: nan
Epoch 10/100
242/242 [==============================] - 0s 914us/step - loss: nan - val_loss: nan
Epoch 11/100
121/121 [=

RuntimeError: Cannot clone object <tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7fd490221550>, as the constructor either does not set or modifies parameter learning_rate

In [16]:
rand_search_cv.best_params_

{'learning_rate': 0.0053800236651201615, 'n_layers': 3, 'n_units': 18}

In [17]:
rand_search_cv.best_score_

-0.3387649754683177